![logo](https://github.com/donatellacea/DL_tutorials/blob/main/notebooks/figures/1128-191-max.png?raw=true)

# XAI in deep learning-based image analysis 

## Part 2: Applying Grad-CAM

---

In the second part of this tutorial we are going to discover Grad-CAM, a gradient-based pixel attribution method for  understanding which parts of the image the model considers more relevant when making its prediction. We will again be using the pre-trained ResNet-50 and play around with various images.

### Setup Colab environment

If you installed the packages and requirements on your own machine, you can skip this section and start from the import section.
Otherwise you can follow and execute the tutorial on your browser. In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HelmholtzAI-Consultants-Munich/XAI-Tutorials/blob/ml4earth-Hackathon/xai-model-specific/Grad-CAM/part2.ipynb)

Now that you are visualizing the notebook in Colab, run the next cell, in order to create a folder in your Google Drive. All the files for this tutorial will be uploaded to this folder. After the first execution you might receive some warning and notification, please follow these instructions:
1. Warning: This notebook was not authored by Google. *Click* on 'Run anyway'.
2. Permit this notebook to access your Google Drive files? *Click* on 'Yes', and select your account.
3. Google Drive for desktop wants to access your Google Account. *Click* on 'Allow'.

At this point, a folder has been created and you can navigate it through the lefthand panel in Colab, you might also have received an email that informs you about the access on your Google Drive. 

In [ ]:
# Create a folder in your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive

In [ ]:
# Don't run this cell if you already cloned the repo in the first part of the tutorial
!git clone https://github.com/HelmholtzAI-Consultants-Munich/XAI-Tutorials.git 

In [ ]:
%cd XAI-Tutorials/xai-model-specific/Grad-CAM

### Import

In [ ]:
# Installing the necessary packages
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import cv2 as cv

from tools.gradcam_model import GradCamModel

## Grad-CAM: theoretical part

Before starting with the hands-on part, click on the next image and watch a video that summarizes the theory behind the Grad-CAM method.



In [ ]:
from IPython.display import VimeoVideo

VimeoVideo("745319946?h=fcd327fc80", width=800, height=600)

## Grad-CAM: hands-on

### Loading and preparing the input image

We need to perform the following steps to have the input image ready to be fed to the model:

* Load the image
* Convert the image to RGB channel
* Transform the image -
  * Convert to PIL format
  * Resize to lower resolution
  * Convert to tensor dtype
  * Normalize the pixel values
* Unsqueeze the image to add a batch dimension


In [ ]:
def read_img(path_to_img):
    img = cv.imread(path_to_img) # Insert the path to image.
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return img

# define the transformations (here we are using the ImageNet transformations)
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize((224, 224)), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

def transform_img(img, transform):
    arr_img = np.array(img)
    # apply the transforms
    trans_img = transform(arr_img)
    # unsqueeze to add a batch dimension
    trans_img = trans_img.unsqueeze(0)
    return trans_img

In [ ]:
# read and visualize the image
path_to_img = 'images/Iguana.jpeg'
img = read_img(path_to_img)
plt.imshow(img)
plt.show()

# next transform the so it is ready to be given to the model
trans_img = transform_img(img, transform)
print('Transformed image size: ', trans_img.size())

### Loading a Grad-Cam model

For this tutorial, we are going to use the GradCamModel class which includes a pretrained ResNet-50 model and functions for returning the feature map activations and gradients. If you are interested you will find this class in ```gradcam_model.py```.

Let's start loading the model:

In [ ]:
# init the GradCamModel
gcmodel = GradCamModel()

# set the evaluation mode
_ = gcmodel.eval()

### Forward pass

After instantiating GradCamModel, we make a forward pass through the network and see what the ResNet-50 predicts. 

In [ ]:
# forward pass
out = gcmodel(trans_img) 
pred = out.argmax(dim=1)

print('Classification result:', pred.item())

Click [this link](https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/) to identify the class name and see if the prediction is correct. 

Now that we verified that our model correctly classifies the iguana, we are going to do the back-propagation with the logit of the predicted class in the ImageNet dataset.

Grad-CAM uses the feature maps produced by the last convolutional layer of a CNN. The authors of Grad-CAM argue, _“we can expect the last convolutional layers to have the best compromise between high-level semantics and detailed spatial information.”_

Grad-CAM is applied to a neural network that has finished training. The weights of the neural network are fixed. We feed an image into the network to calculate the Grad-CAM heatmap for a specific image and a chosen class of interest.

### Grad-CAM Details

Let us assume $y^c$ is the score for class $c$ i.e., the output for class $c$ before the softmax.

**Step 1: Computing Gradient**

Compute the gradient of $y^c$ with respect to the feature map activation $A^k$ of a convolution layer i.e., $\frac {∂y^c}{∂A^k}$

**Step 2: Calculate Global Average Pooling (GAP) of the feature map.**
     
Global average pool the gradients over the width dimension (indexed by $i$) and the height dimension (indexed by $j$) to obtain weights ${\alpha_k^c}$

  $
  {\alpha_k^c} = \frac {1}{Z} \sum_{i} \sum_{j} \frac {∂y^c}{∂A^k_{ij}}
  $

**Step 3: Calculate Final Grad-CAM Localization Map**
     
Perform a weighted combination of the feature map activations $A^k$ where the weights are the ${\alpha_k^c}$ we just calculated and keep only positive contributions applying a ReLU function.

  $
  L^c_{Grad-CAM} = ReLU (\sum_k {\alpha_k^c} A^k)
  $

#### Exercise 1
Follow the steps described above and change the **Nones** with your code to complete the Grad-CAM algorithm.

HINTS: 

1. Don't forget the batch dimension - gradients will have a size of $[Batch \times k \times i \times j]$
2. Use numpy to apply the ReLU function

In [ ]:
def calculate_localization_map(gcmodel, img, out, c):
    
    # Step 1 - Gradient output y wrt. to activation map 
    # get the gradient of the output with respect to the parameters of the model
    out[:,c].backward(retain_graph=True)
    # pull the gradients out of the model
    gradients = gcmodel.get_gradient()

    # Step 2 - Global average pooling
    # pool the gradients across the channels
    pooled_gradients = torch.mean(None, dim=None)

    # Step 3 - Weighted combination of influence and feature maps
    # get the activations of the last convolutional layer
    activations = gcmodel.get_activations(img).detach()    
    # weight the channels by corresponding gradients
    for i in range(activations.size(1)):
        activations[:, i, :, :] *= pooled_gradients[i]
    # average the channels of the activations
    localization_map = torch.sum(activations, dim=1).squeeze()
    # convert the map to be a numpy array
    localization_map = localization_map.numpy()
    # relu on top of the localization map
    localization_map = None
    
    return localization_map

def convert_to_heatmap(localization_map, img):
    # normalize the localization_map
    localization_map /= np.max(localization_map)
    # resize to image size
    heatmap = cv.resize(localization_map, (img.shape[1], img.shape[0]))
    # normalize to [0, 255] range and convert to unsigned int
    heatmap = np.uint8(255 * heatmap)
    return heatmap


### Heatmap

Call calculate_heatmap to get the Res-Net GradCAM output for class 'iguana'

Next, process the result for visualisation and visualise it along with the original image and the Grad-CAM heat-map projected onto the original image.

In [ ]:
localization_map = calculate_localization_map(gcmodel, trans_img, out, c=39)

heatmap = convert_to_heatmap(localization_map, img)

plt.figure(figsize=(15, 15))
plt.subplot(131)
plt.imshow(img)
plt.title('Original image')
plt.subplot(132)
plt.imshow(localization_map.squeeze())
plt.title('Heat-Map for the image')
plt.subplot(133)
plt.imshow(heatmap, cmap='jet', alpha=0.5)
plt.imshow(img, alpha=0.5)
plt.title('Grad-CAM heat-map projected onto the original image')
plt.show()

### Now let's look at a different image

In [ ]:
# read and visualize an image
img = read_img('images/cat_and_dog.jpg')
plt.imshow(img)
plt.show()

# next transform it so it is ready to be given to the model
trans_img = transform_img(img, transform)
print('Transformed image size: ', trans_img.size())

# forward pass
out = gcmodel(trans_img) # img)
pred = out.argmax(dim=1)

print('Classification result:', pred.item())

What is the model predicting? Let's look at the top two predictions and try and understand each!

In [ ]:
pred_sort= torch.argsort(out, descending=True)
top_ten = pred_sort[0,:10]
print('Top ten probabilities: ', top_ten)

Take a moment to identify the class names [ImageNet class list](https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/).

Now let's use GradCAM to try and understand where the model is focusing for specific probabilities

In [ ]:
heatmap_cat = calculate_localization_map(gcmodel, trans_img, out, c=282)
heatmap_dog = calculate_localization_map(gcmodel, trans_img, out, c=231)

heatmap_catn = convert_to_heatmap(heatmap_cat, img)
heatmap_dogn = convert_to_heatmap(heatmap_dog, img)

plt.figure(figsize=(15, 15))
plt.subplot(141)
plt.imshow(heatmap_cat.squeeze())
plt.title('Heatmap for class  garden cat')
plt.subplot(142)
plt.imshow(heatmap_catn, cmap='jet', alpha=0.4)
plt.imshow(img, alpha=0.6)
plt.title('Grad-CAM cat projection')
plt.subplot(143)
plt.imshow(heatmap_dog.squeeze())
plt.title('Heatmap for class  dog')
plt.subplot(144)
plt.imshow(heatmap_dogn, cmap='jet', alpha=0.4)
plt.imshow(img, alpha=0.6)
plt.title('Grad-CAM dog projection')
plt.show()

---
Congratulations! You completed this tutorial.

## Questionnaire
What did we learn today?

Look at [this short questionnaire](https://docs.google.com/presentation/d/1vd_HfkBD4FokoAM2el4T1rXWnD7_0FAEvxk4HgiYjXs/present?slide=id.g1499f1249e3_0_0) to revise what you have learned in our session today!

## Homework

* Look at the top-10 classes of the model - can you see a pattern?
* Can you implement the occlusion technique, see [here](https://towardsdatascience.com/inshort-occlusion-analysis-for-explaining-dnns-d0ad3af9aeb6), by removing the areas of the dog and cat on the last image and see what you get?

## Further reading:

Here is the original GradCAM paper: https://arxiv.org/pdf/1610.02391.pdf

This tutorial was based on: https://medium.com/@stepanulyanin/grad-cam-for-resnet152-network-784a1d65f3